In [5]:
import numpy as np

In [30]:
S = 100
K = 95.
r = .08
v = .3 ### standard deviation
q = 0 ### dividend payouts
expiry = 1 ### T in notes
n = 2
h = expiry/n
u = np.exp((r-q)*h+v*np.sqrt(h))
d = np.exp((r-q)*h-v*np.sqrt(h))

In [31]:
print(u)
print(d)

1.2867659233791633
0.8418680103294537


In [32]:
def call_payoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

def put_payoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

In [33]:
### No-Arbitrage form
fu = call_payoff(u * S, K)
fd = call_payoff(d * S, K)
D = (fu - fd)/(S * (u - d))                        ## Delta
B = np.exp(-r * h) * ((u * fd - d * fu)/ (u - d))  ## B 
f_no_arb = S * D + B
f_no_arb

14.46852100429276

In [34]:
### Risk-neutral form
pstar = (np.exp((r - q)*h) - d)/(u - d)
f_risk_n = np.exp(-r*h)*(fu*pstar - fd*(1-pstar))
f_risk_n

14.468521004292745

In [35]:
### Single Period Binomial Function Model
def single_period_binom_model(S, K, r, v, T, n, payoff):
    h = T/n
    u = np.exp((r-q)*h+v*np.sqrt(h))
    d = np.exp((r-q)*h-v*np.sqrt(h))
    fu = payoff(u*S, K)
    fd = payoff(d*S, K)
    pstar = (np.exp((r - q)*h) - d)/(u - d)
    f0 = np.exp(-r*h)*(fu*pstar + fd*(1-pstar))
    
    return f0

In [36]:
call_price_1 = single_period_binom_model(S, K, r, v, expiry, n, call_payoff)

In [37]:
put_price_1 = single_period_binom_model(S, K, r, v, expiry, n, put_payoff)

In [38]:
call_price_1

14.468521004292745

In [39]:
put_price_1

5.743517723763451

In [40]:
put_payoff(S, K)

0.0

In [42]:
S = 100.0
K = 95.0
r = 0.08
v = 0.30
q = 0.0
T = 1
n = 2
h = T / n
nodes = n + 1
spot_T = np.zeros((nodes, ))
spot_T

array([0., 0., 0.])

In [43]:
for i in range(nodes):
    spot_T[i] = S*(u**(i))*(d**(n-i))
spot_T

array([ 70.87417468, 108.32870677, 165.57665416])

In [44]:
from scipy.stats import binom

In [45]:
probs = np.zeros((nodes, ))
for i in range(nodes):
    probs[i] = binom.pmf(i,n,pstar)
probs

array([0.30562657, 0.49441692, 0.19995651])

In [46]:
call_T = call_payoff(spot_T, K)
call_T

array([ 0.        , 13.32870677, 70.57665416])

In [49]:
call_price_rn = np.exp(-r * expiry) * np.dot(call_T, probs)
call_price_rn

19.11053913600299

In [57]:
def euro_binomial_pricer(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    
    price = 0.0
    
    for i in range(nodes):
        prob = binom.pmf(i, n, pstar)
        spotT = S * (u ** i) * (d ** (n - i))
        po = payoff(spotT, K) 
        price += po * prob
        if verbose:
            print(f"({spotT:0.4f}, {po:0.4f}, {prob:0.4f})")
        
    price *= np.exp(-r * T)
    
    return price

In [58]:
euro_binomial_pricer(S, K, r, v, q, expiry, n, call_payoff, verbose = True)

(26.4157, 0.0000, 0.1603)
(37.3513, 0.0000, 0.4044)
(52.8140, 12.8140, 0.3400)
(74.6781, 34.6781, 0.0953)


7.073853261277715

In [59]:
euro_binomial_pricer(S, K, r, v, q, expiry, n, put_payoff, verbose = True)

(26.4157, 13.5843, 0.1603)
(37.3513, 2.6487, 0.4044)
(52.8140, 0.0000, 0.3400)
(74.6781, 0.0000, 0.0953)


2.998507116743151

In [60]:
### recursive model for European binomial pricer
### code from Dr. Brough

In [63]:
S = 41
K = 40
r = .08
v = .3
q = 0
expiry = 1
n = 3
put_price = euro_binomial_pricer(S, K, r, v, q, expiry, n, put_payoff, verbose = True)
print(f"The European Put Premium: ${put_price:0.3f}")

(26.4157, 13.5843, 0.1603)
(37.3513, 2.6487, 0.4044)
(52.8140, 0.0000, 0.3400)
(74.6781, 0.0000, 0.0953)
The European Put Premium: $2.999


In [74]:
def euro_binomial_pricer_recursive(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0-pu
    disc = np.exp(-r*h)
    
    ### Arrays to store the spot prices and option values
    Ct = np.empty(nodes)
    St = np.empty(nodes)
    
    for i in range(nodes):
        St[i] = S * (u ** (n - i) * (d ** (i)))
        Ct[i] = payoff(St[i], K)
    
    ###(start at n-1 column, go to but not including -1, go back by steps of -1)
    for t in range((n-1), -1, -1):
        ### go up to but not including step t +1, so it goes up to the penultimate node
        for j in range(t+1):
            Ct[j] = disc*(pd*Ct[j+1] + pu*Ct[j])
            #St[j] = St[j]/u
            #Ct[j] = np.maximum(ct[j], early payoff)
            
            
    return Ct[0]

In [75]:
euro_binomial_pricer_recursive(S, K, r, v, q, expiry, n, put_payoff, verbose = True)

2.9985071167431516